<a href="https://colab.research.google.com/github/beotavalo/geospatial-chat/blob/main/EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install python-docx rapidfuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 25.9 MB/s eta 0:00:00


In [7]:
from docx import Document

def extract_content_from_docx(file_path):
    document = Document(file_path)
    titles, questions, paragraphs = [], [], []
    current_title = None
    current_question = None

    for paragraph in document.paragraphs:
        text = paragraph.text.strip()
        if text:
            # Asume que los títulos y preguntas tienen un estilo particular
            if paragraph.style.name == 'Heading 1':
                current_title = text
                titles.append(current_title)
            elif paragraph.style.name == 'Heading 2':
                current_question = text
                questions.append(current_question)
            else:
                paragraphs.append({
                    'title': current_title,
                    'question': current_question,
                    'text': text
                })
    return titles, questions, paragraphs

# Ejemplo de uso
file_path = '/content/geodocument.docx'
titles, questions, paragraphs = extract_content_from_docx(file_path)

In [8]:
import re

# Definir áreas y palabras clave
areas = {
    "surveying": ["survey", "measurement", "boundary", "land"],
    "mapping": ["map", "cartography", "GIS"],
    "remote sensing": ["satellite", "imagery", "remote sensing"],
    "geospatial intelligence": ["geospatial", "intelligence", "analysis"]
}

def classify_area(paragraph_text):
    for area, keywords in areas.items():
        if any(re.search(rf"\b{keyword}\b", paragraph_text, re.IGNORECASE) for keyword in keywords):
            return area
    return "unknown"

# Añadir el área a cada párrafo
for p in paragraphs:
    p['area'] = classify_area(p['text'])

In [9]:
from rapidfuzz import fuzz, process

def remove_similar_paragraphs(paragraphs, similarity_threshold=85):
    unique_paragraphs = []
    for p in paragraphs:
        is_unique = True
        for unique_p in unique_paragraphs:
            similarity = fuzz.ratio(p['text'], unique_p['text'])
            if similarity > similarity_threshold:
                is_unique = False
                break
        if is_unique:
            unique_paragraphs.append(p)
    return unique_paragraphs

# Eliminar párrafos similares
cleaned_paragraphs = remove_similar_paragraphs(paragraphs)

In [11]:
import pandas as pd

def export_to_csv(paragraphs, output_file='output.csv'):
    data = [{'Pregunta': p['question'], 'Área': p['area'], 'Texto': p['text']} for p in paragraphs]
    df = pd.DataFrame(data)
    df.to_csv(output_file, index=False)

# Guardar el contenido procesado
export_to_csv(cleaned_paragraphs)

In [12]:
import re
from docx import Document

def extract_content_from_docx(file_path):
    document = Document(file_path)
    paragraphs = []
    current_title = None
    current_question = None

    # Búsqueda de preguntas con patrones de signos de interrogación
    question_pattern = re.compile(r'.*\?')

    for paragraph in document.paragraphs:
        text = paragraph.text.strip()
        if text:
            # Si es una pregunta, la identificamos
            if question_pattern.match(text):
                current_question = text
            # Si no es pregunta, se considera como párrafo de texto bajo un título/pregunta
            else:
                paragraphs.append({
                    'title': current_title,
                    'question': current_question,
                    'text': text
                })
                # Reiniciar pregunta una vez añadida
                current_question = None
    return paragraphs

# Ejemplo de uso
file_path = '/content/geodocument.docx'
paragraphs = extract_content_from_docx(file_path)

# Imprimir el resultado
for p in paragraphs:
    print(f"Título: {p['title']}, Pregunta: {p['question']}, Texto: {p['text']}")

Título: None, Pregunta: 1. What is a land survey?, Texto: A land survey is a report in the form of a map showing the location of a parcel of real property. The map will also show the location of visible improvements on and adjacent to the property.
Título: None, Pregunta: 2. Why are land surveys so important?, Texto: Prior to the closing on your new home purchase, a land survey provides important information to you as the buyer. The survey map will show the limits of the land you are purchasing and identify any conflicts in your deed. It will also allow you to see if improvements such as driveways, fences, wells or even dwellings encroach over the property lines. Any existing property corner markers found by the surveyor will also be shown on the map.
Título: None, Pregunta: 3. How is a land survey performed?, Texto: The land surveyor's responsibility is to locate on the ground the boundaries of the land described in the deed. The surveyor also examines and maps various visible and app

In [13]:
import pandas as pd

def export_to_csv(paragraphs, output_file='output.csv'):
    data = [{'Pregunta': p['question'], 'Área': p['area'], 'Texto': p['text']} for p in paragraphs]
    df = pd.DataFrame(data)
    df.to_csv(output_file, index=False)

# Guardar el contenido procesado
export_to_csv(paragraphs)


KeyError: 'area'

In [15]:
import re
from docx import Document
import pandas as pd
from rapidfuzz import fuzz, process

# Definir áreas y palabras clave
areas = {
    "surveying": ["survey", "measurement", "boundary", "land"],
    "mapping": ["map", "cartography", "GIS"],
    "remote sensing": ["satellite", "imagery", "remote sensing"],
    "geospatial intelligence": ["geospatial", "intelligence", "analysis"]
}

def classify_area(paragraph_text):
    for area, keywords in areas.items():
        if any(re.search(rf"\b{keyword}\b", paragraph_text, re.IGNORECASE) for keyword in keywords):
            return area
    return "unknown"

def extract_content_from_docx(file_path):
    document = Document(file_path)
    paragraphs = []
    current_title = None
    current_question = None

    # Búsqueda de preguntas con patrones de signos de interrogación
    question_pattern = re.compile(r'.*\?')

    for paragraph in document.paragraphs:
        text = paragraph.text.strip()
        if text:
            # Si es una pregunta, la identificamos
            if question_pattern.match(text):
                current_question = text
            # Si no es pregunta, se considera como párrafo de texto bajo un título/pregunta
            else:
                paragraphs.append({
                    'title': current_title,
                    'question': current_question,
                    'text': text
                })
                # Reiniciar pregunta una vez añadida
                current_question = None
    return paragraphs


def remove_similar_paragraphs(paragraphs, similarity_threshold=85):
    unique_paragraphs = []
    for p in paragraphs:
        is_unique = True
        for unique_p in unique_paragraphs:
            similarity = fuzz.ratio(p['text'], unique_p['text'])
            if similarity > similarity_threshold:
                is_unique = False
                break
        if is_unique:
            unique_paragraphs.append(p)
    return unique_paragraphs

def export_to_csv(paragraphs, output_file='output.csv'):
    data = [{'Pregunta': p['question'], 'Área': p['area'], 'Texto': p['text']} for p in paragraphs]
    df = pd.DataFrame(data)
    df.to_csv(output_file, index=False)

# Ejemplo de uso
file_path = '/content/geodocument.docx'
paragraphs = extract_content_from_docx(file_path)

# Añadir el área a cada párrafo
for p in paragraphs:
    p['area'] = classify_area(p['text'])

# Eliminar párrafos similares
cleaned_paragraphs = remove_similar_paragraphs(paragraphs)

# Guardar el contenido procesado
export_to_csv(cleaned_paragraphs)